<a href="https://colab.research.google.com/github/Aditi1203/CMPE-295/blob/master/2DCNN_Butterworth_ST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Aditi1203/CMPE-295.git

Cloning into 'CMPE-295'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1152144 (delta 4), reused 0 (delta 0), pack-reused 1152132
Receiving objects: 100% (1152144/1152144), 3.34 GiB | 30.05 MiB/s, done.
Resolving deltas: 100% (220404/220404), done.
Checking out files: 100% (738055/738055), done.


In [2]:
pwd

'/content'

In [3]:
cd /content/CMPE-295/st_petersberg_segments/

/content/CMPE-295/st_petersberg_segments


In [4]:
import pandas as pd
import numpy as np
import cv2
import os
import imutils
from keras.applications import VGG19, VGG16
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from PIL import Image
import keras
import tensorflow as tf
from keras.models import Model, load_model
from keras import Sequential, losses, optimizers, Input
from keras.layers import Dense, Conv2D, MaxPool2D, Average, Flatten, Dropout, Activation
from keras.utils import to_categorical, plot_model,vis_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array
from skimage import io
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pywt
from scipy import signal
from scipy.spatial import distance


Using TensorFlow backend.


In [0]:
class DataModel:
    """
            DataModel
            not used currently in the program.
    """
    def __init__(self):
        self.items_labels = None
        self.images = []
        self.curated_data = {}
    def add_image(self, image_array):
        self.images.append(image_array)
    def add_labels(self, df):
        self.item_labels = df
    def add_curated_data(self, image, emotion):
        self.curated_data['segment'] = image #Array from csv of each segment
        self.curated_data['label'] = label #Label
        self.curated_data['scalogram'] =scaledImage #scalogram of each segment

def load_data(number_of_items=90):
    """
        number_of_items -> Number of items to return
        returns the data in a dictionary of images and labels.
    """
    path = "Scalogram"
    path1= "Scalogram/Cropped"
    data = [] 
    curated_data = {"label":[], "scalogram":[]}
    for subject_name in os.listdir(path)[:number_of_items]:
        # At the start of the iteration build a data model
        data_model = DataModel()
        if subject_name == ".DS_Store":
            continue
        if subject_name  ==".ipynb_checkpoints":
            continue
        print ("Going through subject:" + subject_name)
        base=os.path.basename(path+"/"+subject_name)
        labelData=os.path.splitext(base)[0]
        print(labelData)
        i=0
        for items in os.listdir(path+"/"+subject_name):
            if items == ".DS_Store":
                continue
            if items.endswith(".png"):
                #i=i+1
                #print(i)
                try:
                    im2 = cv2.imread(path+"/"+subject_name+"/"+items)
                    #plt.imshow(im2)
                    #plt.show()
                    crop_img = im2[30:20+235, 50:50+342]
                    im = cv2.resize(crop_img, (224,224)) # Changing into 80x80X3
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                    #print(type(crop_img))
                    #plt.imshow(crop_img)
                    #plt.show()
                    

              #curated_data['segments'].append(df1)
                    #curated_data['scalogram'].append(crop_img)
                    curated_data['scalogram'].append(im)
                    curated_data['label'].append(labelData)
              #print(df1)
                except:
                      df = None  
        #data.append(data_model) # Save all the data

    return curated_data


In [6]:
data = load_data()

Going through subject:person_I54
person_I54
Going through subject:person_I18
person_I18
Going through subject:person_I09
person_I09
Going through subject:person_I44
person_I44
Going through subject:person_I68
person_I68
Going through subject:person_I03
person_I03
Going through subject:person_I20
person_I20
Going through subject:person_I25
person_I25
Going through subject:person_I38
person_I38
Going through subject:person_I33
person_I33
Going through subject:person_I12
person_I12
Going through subject:person_I35
person_I35
Going through subject:person_I40
person_I40
Going through subject:person_I08
person_I08
Going through subject:person_I01
person_I01
Going through subject:person_I65
person_I65
Going through subject:person_I49
person_I49
Going through subject:person_I23
person_I23
Going through subject:person_I47
person_I47
Going through subject:person_I42
person_I42
Going through subject:person_I72
person_I72
Going through subject:person_I70
person_I70
Going through subject:person_I57

In [7]:
temp=np.array(data['scalogram'])
print(temp.shape)

(50121, 224, 224)


In [8]:
encoder = LabelEncoder()
encoder.fit(data['label'])

LabelEncoder()

In [0]:
data['label'] = encoder.transform(data['label'])

In [0]:
all_the_classes = encoder.classes_
mapping = {
    0: 'person_I01',
    1: 'person_I03',
    2: 'person_I06',
    3: 'person_I08',
    4: 'person_I09',
    5: 'person_I12',
    6: 'person_I15',
    7: 'person_I16',
    8: 'person_I18',
    9: 'person_I20',
    10: 'person_I23',
    11: 'person_I25',
    12: 'person_I27',
    13: 'person_I29',
    14: 'person_I33',
    15: 'person_I35',
    16: 'person_I38',
    17: 'person_I40',
    18: 'person_I42',
    19: 'person_I44',
    20: 'person_I47',
    21: 'person_I49',
    22: 'person_I51',
    23: 'person_I54',
    24: 'person_I57',
    25: 'person_I59',
    26: 'person_I62',
    27: 'person_I65',
    28: 'person_I68',
    29: 'person_I70',
    30: 'person_I72',
    31: 'person_I74'
} 


#print(mapping)


# Get emotion from class number   
def get_name_from_class(class_number):
    """
        gets the corresponding subject from the class
    """
    if mapping.get(class_number,None):
        return mapping.get(class_number)
    else:
        return -1 # No such class

In [0]:
# Extracting features and labels
features = np.array(data['scalogram'])
labels = data['label']

In [12]:
print ("Shape of features: ", features.shape)
print ("Shape of labels: ", labels.shape)

Shape of features:  (50121, 224, 224)
Shape of labels:  (50121,)


In [0]:
# Changing
labels = to_categorical(labels, num_classes=len(mapping))

In [0]:
# Building model
model = Sequential()
model.add(Conv2D(64, kernel_size=5, input_shape=(224, 224, 1), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=5, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(150, activation="relu"))
model.add(Dense(120, activation="relu"))
model.add(Dense(100, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(80, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(len(mapping), activation="softmax"))

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 220, 220, 64)      1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 106, 106, 64)      102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 53, 53, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 179776)           

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizers.RMSprop(),metrics=['accuracy'])

In [0]:
# Splitting
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [0]:
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.25, random_state=1)

In [0]:
tensorboard = TensorBoard()
earlystopping = EarlyStopping(patience=3)

In [0]:
# Scaling
features_train = features_train.astype("float32")

In [0]:
features_test = features_test.astype("float32")

In [0]:
features_val = features_val.astype("float32")

In [0]:
features_train = features_train / 1/255

In [0]:
features_test = features_test / 1/255

In [0]:
features_val = features_val / 1/255

In [0]:
features_train = features_train.reshape(len(features_train), 224, 224, 1)

In [0]:
features_test = features_test.reshape(len(features_test), 224, 224, 1)

In [0]:
features_val = features_val.reshape(len(features_val), 224, 224, 1)

In [29]:
# Training
model.fit(features_train, labels_train, epochs=100, batch_size=32, callbacks=[tensorboard,earlystopping], validation_data=(features_test, labels_test))

Train on 30072 samples, validate on 10025 samples
Epoch 1/100
30072/30072 [==============================] - 61s 2ms/step - loss: 2.1597 - accuracy: 0.4367 - val_loss: 1.5425 - val_accuracy: 0.5515
Epoch 2/100
30072/30072 [==============================] - 53s 2ms/step - loss: 1.0771 - accuracy: 0.6771 - val_loss: 0.9482 - val_accuracy: 0.7243
Epoch 3/100
30072/30072 [==============================] - 53s 2ms/step - loss: 0.8024 - accuracy: 0.7597 - val_loss: 0.8040 - val_accuracy: 0.7684
Epoch 4/100
30072/30072 [==============================] - 53s 2ms/step - loss: 0.6007 - accuracy: 0.8153 - val_loss: 0.5824 - val_accuracy: 0.8277
Epoch 5/100
30072/30072 [==============================] - 53s 2ms/step - loss: 0.4793 - accuracy: 0.8523 - val_loss: 0.4460 - val_accuracy: 0.8618
Epoch 6/100
30072/30072 [==============================] - 53s 2ms/step - loss: 0.4187 - accuracy: 0.8702 - val_loss: 0.5482 - val_accuracy: 0.8315
Epoch 7/100
30072/30072 [==============================] - 53s

In [0]:
model.save("2DCNN_ButterworthKalmann_ST.h5") # Saving the model

In [31]:
pwd

'/content/CMPE-295/st_petersberg_segments'

In [0]:
# create authentication model with base model
base_model = keras.models.load_model('2DCNN_ButterworthKalmann_ST.h5')
base_model.layers.pop()
authenticate_model = Model(base_model.input, base_model.layers[-1].output)
for layer in authenticate_model.layers: 
  layer.trainable = False

# k_seg = []
# k_seg_prediction= []
# person_label = get_emotion_from_class(np.argmax(labels_val[300]))
   
# taking an element from K_seg to act as current segment
# person_107 = k_seg[0]
# k_seg = k_seg[1:]
# print(person_107) 
# auth_prediction = authenticate_model.predict(person_107.reshape(1,224,224,1))[0]


def dissimilarity_calc(person):
  # print(current_vector)
  # print(current_vector.shape)
  k_seg_vector = prediction_dict[person]
  dists = distance.cdist(k_seg_vector, k_seg_vector, 'euclidean')
  
  max_dist = []
  for i in range(len(dists)):
    mean=np.mean(dists[i])
    standard_deviation=np.std(dists[i])
    dist_mean=abs(dists[i]-mean)
    soln_array=dist_mean<(standard_deviation)
    soln_array1=dists[i][soln_array]
    max_dist.append(max(soln_array1))

  average = sum(max_dist)/len(max_dist)
  print("threshold for ",person," is:",average)
  return(average)



In [0]:
def authenticate_user(username, path_to_scalogram, threshold = 0.0003):
  username_vector_list = []
  for i in range(len(features_train)):
    if get_emotion_from_class(np.argmax(labels_train[i])) == username:
      username_vector_list.append(authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0])
  #get image data
  # print(username_vector_list)
  im = cv2.imread(path_to_scalogram)
  crop_img = im[30:20+235, 50:50+342]
  im = cv2.resize(crop_img, (224,224)) # Changing into 80x80X3
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im = im.astype('float32')
  im = im / 1/255
  scalogram_vector = authenticate_model.predict(im.reshape(1,224,224,1))[0]
  # print(scalogram_vector)
  dissimilarity_score = dissimilarity_calc(scalogram_vector, username_vector_list)
  # print(dissimilarity_score)
  if dissimilarity_score <= threshold:
    print ("Authenticated with dissimilarity score: ", dissimilarity_score)
  else:
    print ("Not Authenticated due to high dissimilarity: ", dissimilarity_score)




In [34]:
#Segregate segments based on the subject label
def create_map():
  map = {}
  for i in range(len(features_train)):
    print(i)
    if get_emotion_from_class(np.argmax(labels_train[i])) in map.keys():
       map[get_emotion_from_class(np.argmax(labels_train[i]))].append(authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0])
    else:
      map[get_emotion_from_class(np.argmax(labels_train[i]))] = []
      map[get_emotion_from_class(np.argmax(labels_train[i]))].append(authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0])
  return map
   
prediction_dict = create_map()
# print(map['person_100'])

Streaming output truncated to the last 5000 lines.
25073
25074
25075
25076
25077
25078
25079
25080
25081
25082
25083
25084
25085
25086
25087
25088
25089
25090
25091
25092
25093
25094
25095
25096
25097
25098
25099
25100
25101
25102
25103
25104
25105
25106
25107
25108
25109
25110
25111
25112
25113
25114
25115
25116
25117
25118
25119
25120
25121
25122
25123
25124
25125
25126
25127
25128
25129
25130
25131
25132
25133
25134
25135
25136
25137
25138
25139
25140
25141
25142
25143
25144
25145
25146
25147
25148
25149
25150
25151
25152
25153
25154
25155
25156
25157
25158
25159
25160
25161
25162
25163
25164
25165
25166
25167
25168
25169
25170
25171
25172
25173
25174
25175
25176
25177
25178
25179
25180
25181
25182
25183
25184
25185
25186
25187
25188
25189
25190
25191
25192
25193
25194
25195
25196
25197
25198
25199
25200
25201
25202
25203
25204
25205
25206
25207
25208
25209
25210
25211
25212
25213
25214
25215
25216
25217
25218
25219
25220
25221
25222
25223
25224
25225
25226
25227
25228
25229
25230
2

In [38]:
# cell for calculating average threshold

threshold_dict = {}
for person in prediction_dict:
  threshold_dict[person] = dissimilarity_calc(person)


threshold for  person_I33  is: 0.14788788709714112
threshold for  person_I12  is: 0.6212440317928359
threshold for  person_I51  is: 0.8652287159798101
threshold for  person_I65  is: 0.4618137533215169
threshold for  person_I16  is: 0.6324874351569933
threshold for  person_I40  is: 0.489226891131057
threshold for  person_I01  is: 0.38261015053249064
threshold for  person_I09  is: 0.3053015373997154
threshold for  person_I54  is: 0.253636455594351
threshold for  person_I35  is: 0.4377453285051885
threshold for  person_I62  is: 0.7870863792232555
threshold for  person_I49  is: 0.6247214350769034
threshold for  person_I03  is: 0.4156629048214345
threshold for  person_I18  is: 0.5489452563370137
threshold for  person_I38  is: 0.43537675459454767
threshold for  person_I20  is: 0.3505847658359407
threshold for  person_I44  is: 0.6360173764250682
threshold for  person_I08  is: 0.7574123130268408
threshold for  person_I68  is: 0.38685403812871744
threshold for  person_I47  is: 0.964481321785678

In [0]:
def similar_score(test_template, stored_templates):
  dists = distance.cdist(test_template, stored_templates, 'euclidean')
  soln=min(dists[0])
  return soln

In [40]:
tp = 0
tn = 0
fp = 0
fn = 0

for i in range(len(features_test)):
  true_label = get_emotion_from_class(np.argmax(labels_test[i]))
  current_segment = authenticate_model.predict(features_test[i].reshape(1,224,224,1))[0]
  for person in prediction_dict:
    test_distance = similar_score(current_segment.reshape(1,current_segment.shape[0]),prediction_dict[person])
    if test_distance <= threshold_dict[person]:
      if person == true_label:
        tp += 1
      else:
        fp += 1
    else:
      if person == true_label:
        fn += 1
      else:
        tn += 1
  print(true_label)
print('tp = ',tp)
print('tn = ',tn)
print('fp = ',fp)
print('fn = ',fn)
accuracy = ((tp+tn)/(tp+fp+tn+fn))*100
print('accuracy:',accuracy)

false_acceptance = ((fp)/(fp+fn))
print('false_acceptance:',false_acceptance)

false_rejection = fn/(tp+tn)
print('false_rejection:',false_rejection)




Streaming output truncated to the last 5000 lines.
person_I38
person_I40
person_I09
person_I12
person_I08
person_I15
person_I23
person_I23
person_I18
person_I54
person_I57
person_I06
person_I44
person_I15
person_I20
person_I62
person_I51
person_I54
person_I33
person_I35
person_I38
person_I27
person_I38
person_I33
person_I49
person_I44
person_I47
person_I62
person_I49
person_I62
person_I01
person_I65
person_I15
person_I09
person_I65
person_I01
person_I65
person_I23
person_I09
person_I25
person_I01
person_I72
person_I44
person_I59
person_I72
person_I62
person_I01
person_I70
person_I33
person_I38
person_I25
person_I23
person_I03
person_I09
person_I33
person_I08
person_I57
person_I15
person_I15
person_I09
person_I59
person_I38
person_I49
person_I42
person_I72
person_I15
person_I38
person_I12
person_I57
person_I40
person_I42
person_I01
person_I72
person_I42
person_I35
person_I18
person_I47
person_I62
person_I57
person_I40
person_I23
person_I03
person_I27
person_I12
person_I40
person_I42
per

In [0]:
subject = "person_200"
claim = "person_201"

# finding prediction vector of the first segment of the subject's test data
for i in range(1,len(features_test)):
  if get_emotion_from_class(np.argmax(labels_test[i])) == subject:
    current_segment_prediction = authenticate_model.predict(features_test[i].reshape(1,224,224,1))[0]
    break
score=similar_score(current_segment_prediction.reshape(1,48), map[claim])
print("threshold for ",claim,":",threshold_dict[claim])
if score>threshold_dict[claim]:
  print("You are not authenticated with dissimarity score:{}".format(score))
else:
  print("Authentication Successful! with dissimarity score:{}".format(score))

threshold for  person_201 : 0.470854108331471
You are not authenticated with dissimarity score:1.1754314537884056


In [0]:
#Map to store all the throshold values per subject
threshold_map = {}

In [0]:
def calc_accuracy():
  for i in range(len(features_train)):
    subject_name = get_emotion_from_class(np.argmax(labels_train[i]))
    subject_vector = authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0]
    username_vector_list = map[subject_name]
    username_vector_list = authenticate_model.predict(np.array(username_vector_list))
    dissimilarity_score = dissimilarity_calc(subject_vector, username_vector_list)
    if subject_name in threshold_map.keys():
      threshold_map[subject_name] = threshold_map[subject_name] + dissimilarity_score
    else:
      threshold_map[subject_name] = dissimilarity_score
    print(i, subject_name, dissimilarity_score)
  

In [0]:
 for j in threshold_map.keys():
    threshold_map[j] = threshold_map[j]/len(map[j])

In [0]:
calc_accuracy()

Streaming output truncated to the last 5000 lines.
952 person_217 0.14147688014484358
953 person_217 0.17223862063033618
954 person_221 0.07734964221868865
955 person_231 0.25184230889526277
956 person_212 0.07223285462811378
957 person_213 0.07240019718066172
958 person_102 0.17191618551465018
959 person_112 0.09080680797267147
960 person_234 0.1595835856127715
961 person_212 0.058295171436516795
962 person_119 0.08184269934656441
963 person_219 0.02453938101823615
964 person_214 0.03588929848838611
965 person_207 0.14417526924768495
966 person_213 0.072913967721018
967 person_230 0.09312851931171034
968 person_101 0.14585786687525018
969 person_220 0.1580139955052798
970 person_102 0.10915267415167125
971 person_116 0.08157692133891291
972 person_104 0.1353370929296338
973 person_219 0.015877886815426803
974 person_214 0.0385428392086002
975 person_215 0.026517189253573797
976 person_113 0.11097605870536145
977 person_217 0.15963878477329976
978 person_111 0.06167889095727308
979 per

KeyboardInterrupt: ignored

In [0]:
#Calculate TP and TN for accuracy


In [0]:
authenticate_user('person_101', 'Scalogram/person_100/seg20.csv.png')

Not Authenticated due to high dissimilarity:  0.0012213722425328595


In [0]:
# Test an image
def test_image(image,label):
    #print(image.shape)
    prediction = np.argmax(model.predict(image.reshape(1,224,224,1)))
    result_predict=get_emotion_from_class(prediction)
    actual=np.argmax(label)
    result_actual=get_emotion_from_class(actual)
    #print(prediction)
    #print(actual)
    return result_actual,result_predict

    
# Get emotion from class number   
def get_emotion_from_class(class_number):
    """
        gets the corresponding label from the class
    """
    if mapping.get(class_number,None):
      return mapping.get(class_number)
    else:
      return -1 # No such class

In [0]:
countFP=0
for i in range(len(features_val)):
    actual,predicted=test_image(features_val[i],labels_val[i])
    print("Model predicts: "+predicted+" and actual label is: "+actual)
    if(predicted!=actual):
        print("Wrong prediction")
        countFP=countFP+1
    print("-------------------------------------------------------------")
  
print("Total False Positives: "+str(countFP))
print("Total Positives: "+str(len(features_val)))